In [5]:
import tweepy
import pandas as pd
import csv

### Tweeter keys:

In [6]:
consumer_key = ''
consumer_secret = ''
access_token = ''
access_token_secret = ''

### Get Twitter IDs:

In [19]:
df = pd.read_csv('data/NAACL_SRW_2016.csv')
df.columns = ['id', 'label']
df = df.mask(df.eq('none')).dropna()

In [20]:
twitter_ids = df.id.tolist()

In [21]:
len(twitter_ids)

5347

### Get Twitter text:

In [22]:
def lookup_tweets(tweet_IDs, api):
    full_tweets = []
    tweet_count = len(tweet_IDs)
    try:
        for i in range((tweet_count // 100) + 1):
            # Catch the last group if it is less than 100 tweets
            end_loc = min((i + 1) * 100, tweet_count)
            full_tweets.extend(
                api.statuses_lookup(tweet_IDs[i * 100:end_loc])
            )
        return full_tweets
    except tweepy.TweepError:
        print ('Something went wrong, quitting...')


auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

# do whatever it is to get por.TweetID - the list of all IDs to look up

tweets = lookup_tweets(twitter_ids, api)

In [25]:
id_text_path = 'data/id_text.tsv' 

In [26]:
with open(id_text_path, 'w') as output_file:
    writer = csv.writer(output_file, delimiter='\t')
    
    for tweet in tweets:
        writer.writerow([tweet.id, tweet.text])

### Create text - labes data:

In [27]:
text_df = pd.read_csv(id_text_path, sep='\t', names=['id', 'text'])

In [28]:
text_df.head()

,id,text
0,572332655397629952,These girls are the equivalent of the irritati...
1,572341498827522049,Drasko they didn't cook half a bird you idiot ...
2,572340476503724032,Hopefully someone cooks Drasko in the next ep ...
3,572334712804384768,of course you were born in serbia...you're as ...
4,575949086055997440,#MKR Lost the plot - where's the big Texan wi...


In [29]:
df.head()

,id,label
0,572341498827522049,racism
1,572340476503724032,racism
2,572334712804384768,racism
3,572332655397629952,racism
4,575949086055997440,racism


In [30]:
df2 = pd.merge(df, text_df, on='id', how='inner')

In [31]:
label_text_df = df2[['text', 'label']]

In [32]:
label_text_df.head()

,text,label
0,Drasko they didn't cook half a bird you idiot ...,racism
1,Hopefully someone cooks Drasko in the next ep ...,racism
2,of course you were born in serbia...you're as ...,racism
3,These girls are the equivalent of the irritati...,racism
4,#MKR Lost the plot - where's the big Texan wi...,racism


In [33]:
label_text_df.label.value_counts()

sexism    2906
racism      16
Name: label, dtype: int64

### Save the data to file:

In [34]:
text_label_path = 'data/text_label.tsv'

In [37]:
label_text_df.to_csv(text_label_path, sep='\t', index=False, header=False)    